In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,west island,-12.1568,96.8225,25.99,78,100,6.17,CC,1696862862
1,1,yellowknife,62.4560,-114.3525,5.79,100,40,4.12,CA,1696862607
2,2,lebanon,36.2081,-86.2911,14.74,70,75,5.66,US,1696862864
3,3,bredasdorp,-34.5322,20.0403,15.12,77,90,7.69,ZA,1696862865
4,4,bethel,41.3712,-73.4140,13.59,66,75,3.60,US,1696862865


In [4]:
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 700,
    size = "Humidity",
    scale = 0.75,
    alpha = 0.5,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# We can say ideal weather has 0 cloudiness, temp between 21 to 27 C and wind speed less than 4.5 m/s

# Use loc to locate above specifications
city_data_df2 = city_data_df.loc[(city_data_df["Cloudiness"] == 0) &
                                 (city_data_df["Max Temp"] >= 21) &
                                 (city_data_df["Max Temp"] <= 27) &
                                 (city_data_df["Wind Speed"] < 4.5)]
city_data_df2

# Drop any rows with null values
city_data_df2 = city_data_df2.dropna()

# Display sample data
city_data_df2

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,mandal,25.4500,74.5667,26.15,52,0,3.80,IN,1696862869
107,107,chui,-33.6971,-53.4616,21.32,52,0,2.41,UY,1696862921
129,129,fiambala,-27.6871,-67.6149,24.51,16,0,2.21,AR,1696862931
188,188,khash,28.2211,61.2158,26.59,18,0,4.22,IR,1696862960
211,211,as sanad,34.4628,9.2640,26.14,34,0,2.52,TN,1696862971
237,237,karratha,-20.7377,116.8463,26.68,44,0,1.91,AU,1696862983
310,310,mournies,35.4764,24.0092,26.02,64,0,2.57,GR,1696863016
359,359,tralee,52.2704,-9.7026,22.14,60,0,1.03,IE,1696863035
362,362,chania,35.5122,24.0156,24.78,64,0,2.57,GR,1696863042
380,380,puerto rico,-26.7960,-55.0240,26.32,38,0,1.14,AR,1696863051


In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df2.copy()
hotel_df = hotel_df[["City","Country","Lat","Lng","Humidity"]]
hotel_df
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
12,mandal,IN,25.4500,74.5667,52,
107,chui,UY,-33.6971,-53.4616,52,
129,fiambala,AR,-27.6871,-67.6149,16,
188,khash,IR,28.2211,61.2158,18,
211,as sanad,TN,34.4628,9.2640,34,
237,karratha,AU,-20.7377,116.8463,44,
310,mournies,GR,35.4764,24.0092,64,
359,tralee,IE,52.2704,-9.7026,60,
362,chania,GR,35.5122,24.0156,64,
380,puerto rico,AR,-26.7960,-55.0240,38,


In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel",
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = hotel_df.loc[index,"Lng"]
    latitude = hotel_df.loc[index,"Lat"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mandal - nearest hotel: No hotel found
chui - nearest hotel: Alerces
fiambala - nearest hotel: Cabañas Lunita
khash - nearest hotel: No hotel found
as sanad - nearest hotel: No hotel found
karratha - nearest hotel: Karratha International Hotel
mournies - nearest hotel: Oasis
tralee - nearest hotel: The Ashe Hotel
chania - nearest hotel: Morum
puerto rico - nearest hotel: Suizo
edgewater - nearest hotel: Best Western Edgewater Inn
bovalino - nearest hotel: No hotel found
zahedan - nearest hotel: هتل صالح;بانک ملی


,City,Country,Lat,Lng,Humidity,Hotel Name
12,mandal,IN,25.4500,74.5667,52,No hotel found
107,chui,UY,-33.6971,-53.4616,52,Alerces
129,fiambala,AR,-27.6871,-67.6149,16,Cabañas Lunita
188,khash,IR,28.2211,61.2158,18,No hotel found
211,as sanad,TN,34.4628,9.2640,34,No hotel found
237,karratha,AU,-20.7377,116.8463,44,Karratha International Hotel
310,mournies,GR,35.4764,24.0092,64,Oasis
359,tralee,IE,52.2704,-9.7026,60,The Ashe Hotel
362,chania,GR,35.5122,24.0156,64,Morum
380,puerto rico,AR,-26.7960,-55.0240,38,Suizo


In [30]:
# Configure the map plot
map_plot2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 700,
    size = "Humidity",
    scale = 1,
    alpha = 0.5,
    color = "City",
    # Use hover_cols to add Hotel Name/Country to hover message
    hover_cols = ["Hotel Name","Country"]
)

# Display the map
map_plot2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)